In [ ]:
from src.hopfield import *
from data.alphabet import alphabet_map
import itertools
import numpy as np
import random
import pandas as pd

In [ ]:
def flatten_matrix(matrix):
  return np.array(matrix).flatten()

In [ ]:
def generate_orthogonal_set(data, i):
  combinations = itertools.combinations(data.keys(), r=i)
  avg_dot_prod = []
  for g in combinations:
    group = np.array([ data[key] for key in g ])
    ortho_matrix = group.dot(group.T)
    np.fill_diagonal(ortho_matrix, 0)
    row, _ = ortho_matrix.shape
    avg_dot_prod.append((np.abs(ortho_matrix).sum() / (ortho_matrix.size - row), g))
  return avg_dot_prod

orthogonal_set = generate_orthogonal_set({ key: np.array([value]).flatten() for key, value in alphabet_map.items() }, 4)
best = min(orthogonal_set, key = lambda dot: dot[0])
worst = max(orthogonal_set, key = lambda dot: dot[0])
best


In [ ]:
import plotly.graph_objects as go
import numpy as np
import math

iterations = []
best_ones = []
worst_ones = []
max_iter = 10
for i in range(2, max_iter):
  our_set = generate_orthogonal_set({ key: np.array([value]).flatten() for key, value in alphabet_map.items() }, i)
  print(i, end=' ')
  best = min(our_set, key = lambda dot: dot[0])
  worst = max(our_set, key = lambda dot: dot[0])
  print(best[0], end=' ')
  print(worst[0])
  iterations.append(i)
  best_ones.append(best[0])
  worst_ones.append(worst[0])

fig = go.Figure(data=go.Scatter(x=iterations, y=best_ones, mode='lines+markers', name='Best Ones'))
fig.add_trace(go.Scatter(x=iterations, y=worst_ones,
                    mode='lines+markers',
                    name='Worst Ones'))
fig.update_layout(title="Best and Worst Values per Amount of Letters Saved (5x5)",
                   xaxis_title='Amount of Letters Saved',
                   yaxis_title='Dot Product Value')
fig.show()

fig = go.Figure(data=go.Scatter(x=iterations, y=best_ones, mode='lines+markers', name='Best Ones'))
fig.update_layout(title="Focusing on the Best Ones (5x5)",
                   xaxis_title='Amount of Letters Saved',
                   yaxis_title='Dot Product Value')
fig.show()

amount_combinations=[]
for i in range(2,max_iter):
  amount_combinations.append(math.comb(26,i))
fig = go.Figure(data=[go.Table(header=dict(values=['#(Saved Letters)', '#Combinations']),
                 cells=dict(values=[iterations, amount_combinations]))
                     ])
fig.show()

In [ ]:
alphabet_matrix = [ alphabet_map[key] for key in best[1] ]
bad_alphabet_matrix = [ alphabet_map[key] for key in worst[1] ]
flattened_train_set = [ flatten_matrix(alphmap) for alphmap in alphabet_matrix ]
flattened_bad_train_set = [ flatten_matrix(alphmap) for alphmap in bad_alphabet_matrix ]

In [ ]:
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots

# Create a NumPy array from your matrix
def print_letters(X: List, title = ""):
  fig = make_subplots(rows = 1, cols = len(X))
  colorscale = [[0, 'white'], [1, 'black']]

  for i, letter in enumerate(X):
    X_array = np.array(letter)
    fig.add_trace(go.Heatmap(z=X_array, colorscale=colorscale), row = 1, col = i + 1)
  fig.update_xaxes(showticklabels=False)
  fig.update_yaxes(showticklabels=False)
  fig.update_coloraxes(showscale=False)
  fig.update_layout(dict({ "title": title }))
  fig.show()

def print_list_progress(X: List[List], title = ""):
  fig = make_subplots(rows = len(X), cols = max([len(x) for x in X]))
  colorscale = [[0, 'white'], [1, 'black']]

  for i, row in enumerate(X):
    for j, letter in enumerate(row):
      X_array = np.array(letter)
      heatmap = go.Heatmap(z=X_array, colorscale=colorscale)
      fig.add_trace(heatmap, row = i + 1, col = j + 1)
      
  fig.update_xaxes(showticklabels=False)
  fig.update_yaxes(showticklabels=False)
  fig.update_coloraxes(showscale=False)
  fig.update_layout(dict({ "title": title, "width": 500 }))
  fig.show()


print_letters([np.flipud(letter) for letter in alphabet_matrix], title = "Letras de mejor ortogonalidad")
print_letters([np.flipud(letter) for letter in bad_alphabet_matrix], title = "Letras de peor ortogonalidad")


In [ ]:
def add_noise(matrix, percentage):
  m = np.copy(matrix)
  for i in range(len(matrix)):
    for j in range(len(matrix[0])):
      m[i][j] *= -1 if random.random() < percentage else 1
  return m

noisy_letters = [add_noise(letter, 0.30) for letter in alphabet_matrix]
not_so_noisy = [add_noise(letter, 0.10) for letter in alphabet_matrix]


In [ ]:
hopfield_matrix = generate_hopfield_matrix(flattened_train_set)

print_letters([np.flipud(letter) for letter in noisy_letters], "Letras de consulta p = 0.3")
print_letters([np.flipud(letter) for letter in not_so_noisy], "Letras de consulta p = 0.1")


progresses = [ most_similar_pattern(hopfield_matrix, flatten_matrix(letter), 10) for letter in noisy_letters ]
progresses2 = [ most_similar_pattern(hopfield_matrix, flatten_matrix(letter), 10) for letter in not_so_noisy ]

print_list_progress([[np.flipud(noisy_letters[i]), *[np.flipud(np.reshape(letter[1], [5,5])) for letter in progress[3]]] for i, progress in enumerate(progresses)], title = "Progresos de letras p = 0.3")

print_list_progress([[np.flipud(not_so_noisy[i]),*[np.flipud(np.reshape(letter[1], [5,5])) for letter in progress[3]]] for i, progress in enumerate(progresses2)], title = "Progresos de letras p = 0.1")

temp = most_similar_pattern(hopfield_matrix, flatten_matrix(alphabet_map["I"]), 10)

def print_energy_linegraph(x: List[List], title = "", letras: List = []):
  fig = go.Figure()
  for i, row in enumerate(x):
    fig.add_trace(go.Scatter(y=row, mode='lines+markers', name=letras[i]))
    
  fig.update_layout(dict({ "title": title }))
  fig.show()


print_energy_linegraph([[paso[2] for paso in progress[3]] for progress in progresses], title = "Energía de recuperación", letras=best[1])
print_energy_linegraph([[paso[2] for paso in progress[3]] for progress in progresses2], title = "Energía de recuperación", letras=best[1])


In [ ]:
bad_matrix = generate_hopfield_matrix(flattened_bad_train_set)
bad_noisy = [add_noise(letter, 0.30) for letter in bad_alphabet_matrix]
bad_not_so_noisy = [add_noise(letter, 0.10) for letter in bad_alphabet_matrix]

print_letters([np.flipud(letter) for letter in bad_not_so_noisy], "Letras de consulta p = 0.1")
print_letters([np.flipud(letter) for letter in bad_noisy], "Letras de consulta p = 0.3")

In [ ]:
p1 = [ most_similar_pattern(bad_matrix, flatten_matrix(letter), 10) for letter in bad_noisy ]
p2 = [ most_similar_pattern(bad_matrix, flatten_matrix(letter), 10) for letter in bad_not_so_noisy ]

print_list_progress([[np.flipud(bad_noisy[i]), *[np.flipud(np.reshape(letter[1], [5,5])) for letter in progress[3]]] for i, progress in enumerate(p1)], title = "Progresos de letras p = 0.3")

print_list_progress([[np.flipud(bad_not_so_noisy[i]),*[np.flipud(np.reshape(letter[1], [5,5])) for letter in progress[3]]] for i, progress in enumerate(p2)], title = "Progresos de letras p = 0.1")

temp = most_similar_pattern(hopfield_matrix, flatten_matrix(alphabet_map["I"]), 10)

def print_energy_linegraph(x: List[List], title = "", letras: List = []):
  fig = go.Figure()
  for i, row in enumerate(x):
    fig.add_trace(go.Scatter(y=row, mode='lines+markers', name=letras[i]))
    
  fig.update_layout(dict({ "title": title }))
  fig.show()

print_energy_linegraph([[paso[2] for paso in progress[3]] for progress in p1], title = "Energía de recuperación", letras=worst[1])
print_energy_linegraph([[paso[2] for paso in progress[3]] for progress in p2], title = "Energía de recuperación", letras=worst[1])